# <div align="center">Acoustic Guitar Classifier</div>

<div align="center">Want to know which kind of acoustic guitar you have? If you're not a guitar enthusiast or a beginner, it can be tricky to distinguish between the many guitar types. I have build this simple app prototype which is powered by machine learning to automatically classify images of four types of acoustic guitars. The underlying model can detect two types of western guitars, i.e. the common dreadnought and the larger jumbo, classical guitars and a very specific guitar type called resonator guitar. If you upload a clear and complete picture of your guitar, you increase the chance that the classifier will get it right.</div>

<div align="center">Simply try it out yourself :-) </div>

In [12]:
# To turn your notebook into a webapp, install voila which will only show markdown and outputs and hide the rest

# Install voila library 
#!pip install voila

# Connect voila to this notebook
#!jupyter serverextension enable voila --sys-prefix

In [4]:
# Import only the necessary libraries for deployment
from utils import *
from fastai2.vision.widgets import *

In [5]:
# Set the path to wherever you have stored your trained classifier
path = Path()

In [6]:
# Load the trained classifier
learn_inf = load_learner(path/'export.pkl')

In [7]:
# Define all iPython widgets that you want to use

# We need a button for uploading images
btn_upload = widgets.FileUpload()
# A button to run the classification
btn_run = widgets.Button(description='Classify image')
# Another button to show the other predictions
btn_show = widgets.Button(description='Show others')
# We want to show the uploaded image as an output
out_pl = widgets.Output()
# And finally we need to display the prediction as a label
lbl_pred = widgets.Label()
# Add more labels for the other predictions
lbl_pred_a = widgets.Label()
lbl_pred_b = widgets.Label()
lbl_pred_c = widgets.Label()

In [8]:
# Create a string to expand the label below
lbl_ext = ' guitar'

In [9]:
# Since the run button doesn't do anything by itself, we need to define an event handler 
def on_click_classify(change):
    # Turn the uploaded image into a PIL image file
    img = PILImage.create(btn_upload.data[-1])
    # Clear all previous outputs
    out_pl.clear_output()
    # Display the uploaded image with 224x224 pixels
    with out_pl: display(img.to_thumb(224,224))
    # Run the classifier on the image
    pred,pred_idx,probs = learn_inf.predict(img)
    # Print out the classification results
    lbl_pred.value = f'Prediction: {(pred + lbl_ext).title()}; Probability: {probs[pred_idx]:.06f}'
    # Reset other labels when making a new prediction
    lbl_pred_a.value = ''
    lbl_pred_b.value = ''
    lbl_pred_c.value = ''

In [10]:
# Attach the event handler to the run button
btn_run.on_click(on_click_classify)

In [11]:
# Define an event handler for the show others button
def on_click_show_others(change):
    # Turn the uploaded image into a PIL image file
    img = PILImage.create(btn_upload.data[-1])
    # Run the classifier on the image
    pred,pred_idx,probs = learn_inf.predict(img)
    # Get predicted probabilities for the other classes
    probs = probs[probs!=probs[pred_idx]]
    # Get names for the other classes
    voc = list(learn_inf.dls.vocab)
    voc.remove(pred)
    # Sort probabilities and corresponding vocab indices
    res = sorted([(v,i) for i,v in enumerate(probs)], reverse=True)
    # Print out the predicted probabilities for the three other classes
    lbl_pred_a.value = f'{(voc[res[0][1]] + lbl_ext).title()}: {res[0][0]:.06f}'
    lbl_pred_b.value = f'{(voc[res[1][1]] + lbl_ext).title()}: {res[1][0]:.06f}'
    lbl_pred_c.value = f'{(voc[res[2][1]] + lbl_ext).title()}: {res[2][0]:.06f}'

In [12]:
# Attach the event handler to the show others button
btn_show.on_click(on_click_show_others)

In [13]:
# Put everything together in a virtual box (note that the order of the components matters)
VBox([widgets.Label('Upload a picture of your guitar!'), 
      btn_upload, btn_run, out_pl, lbl_pred, btn_show, lbl_pred_a, lbl_pred_b, lbl_pred_c], 
      layout=Layout(align_items='center'))

For examples click here:
- [Dreadnought guitar](https://www.bing.com/images/search?q=dreadnought+guitar&scope=images&form=QBLH&sp=-1&pq=dreadnought+guitar&sc=6-18&qs=n&sk=&cvid=361D728C992E4FDA98D42C0202504A4C)
- [Jumbo guitar](https://www.bing.com/images/search?q=jumbo%20guitar&qs=n&form=QBIR&scope=images&sp=-1&pq=jumbo%20guitar&sc=8-12&sk=&cvid=B56651D5293C4DE0B54CD4DF290E8906)
- [Classical guitar](https://www.bing.com/images/search?q=classical%20guitar&qs=n&form=QBIR&scope=images&sp=-1&pq=classical%20guitar&sc=8-16&sk=&cvid=2A870A59F623457986FFB86383DB3A08)
- [Resonator guitar](https://www.bing.com/images/search?q=resonator%20guitar&qs=n&form=QBIR&scope=images&sp=-1&pq=resonato%20guitar&sc=8-15&sk=&cvid=0E2335DE50574F0F92BDFE8671087845)